<a href="https://colab.research.google.com/github/sneharc16/DTU-MLR-Assignments-Deep-Learning/blob/main/Project_Deep_Learning_Models_with_PyTorch_and_Transfer_Learning_MLR_DTU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**initial notes**

This is currently a work in progress. Take mnist data set from torrch vision lib then create a data loader for loading the dataset

after data loading, develop a deep learning models using pytorch

(data loader must have commenets as well, which is path which is train test split)

model 1
model with fully connected layers only
(activations are allow, sigmoid softmpa)(no convulutions)
(in channel and out cahnnels must be metnioned after every layer)


model 2
with convulusion layers followed by a fully connected layer
(conv 2d and fully connected)
(same comment wala standard)
(in channel and out cahnnels must be metnioned after every layer)
(write about bias as well)

model 3
model with only convulution layers
(no fully connected layers)



train the model using default train fucntion and with help of customised trainng loop as well

default means = train function of python

look for train loop as well



furthur wrwite a function for loading a saved model, and performing/ fine tuning on that model

apply k full-cross validation

to check the perfomrane of developed model in differenet folds


make a customised data laoder for a dog cat classifier

dont maek this model from scrat, use the model from mnist here for the dog-cat classifier
(this is basics of tranfer learning)

\**Project: Deep Learning Models with PyTorch and Transfer Learning**

**Objective:** Develop three different deep learning models using the MNIST dataset and PyTorch, train them using both a default training function and a custom training loop, and apply transfer learning to a new task.

**Part 1: Data Preparation**

1. Import the MNIST dataset from the torchvision library.
2. Create a DataLoader for loading the dataset. Include comments in your code explaining the path and train/test split.

**Part 2: Model Development**

Develop three different models as follows:

**Model 1: Fully Connected Neural Network**

- Construct a model using only fully connected layers.
- You may use activation functions such as sigmoid and softmax, but do not use convolutional layers.
- After each layer, include comments in your code specifying the number of input and output channels.

**Model 2: Convolutional Neural Network with Fully Connected Layer**

- Construct a model that includes convolutional layers (Conv2D) followed by a fully connected layer.
- After each layer, include comments in your code specifying the number of input and output channels.
- Also, include comments about the bias in each layer.

**Model 3: Convolutional Neural Network**

- Construct a model using only convolutional layers (Conv2D).
- Do not include any fully connected layers in this model.

**Part 3: Model Training**

- Train each model using the default training function provided by PyTorch.
- Also, develop a custom training loop and use it to train each model.
- Compare the performance of the models when trained with the default function versus the custom loop.

**Part 4: Model Evaluation**

- Write a function for loading a saved model and performing fine-tuning on that model.
- Apply k-fold cross-validation to check the performance of the developed models in different folds.

**Part 5: Transfer Learning**

- Create a custom DataLoader for a dog-cat classifier.
- Don't build this model from scratch. Instead, use one of the models trained on the MNIST dataset and fine-tune it for the dog-cat classification task. This is an example of transfer learning.

Remember to follow best practices for deep learning model development, including appropriate use of training/validation/test splits, normalization of input data, and regularization to prevent overfitting. Good luck!

# code below


These are the packages which will be required thorught the project

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import os
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from sklearn.model_selection import KFold

# Data Loading
**Importing Dataset**

I have used in-built data loader of pytorch via which I have divided the data into to train and test as well

In [ ]:
from torchvision import datasets
from torchvision.transforms import ToTensor

train_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
    )

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
    )

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9.91M/9.91M [00:00<00:00, 18.0MB/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28.9k/28.9k [00:00<00:00, 486kB/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.56MB/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4.54k/4.54k [00:00<00:00, 9.69MB/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [ ]:
from torch.utils.data import DataLoader

loaders = {
    'train': torch.utils.data.DataLoader(train_data, batch_size=100, shuffle=True, num_workers=2),
    'test': torch.utils.data.DataLoader(test_data, batch_size=100, shuffle=True, num_workers=2)
}


# Model Developement

**Fully connected Neural Netwrok**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class FullyConnectedModel(nn.Module):
    def __init__(self):
        super(FullyConnectedModel, self).__init__()

        # Input layer: 784 input channels (for a 28x28 image), 512 output channels
        self.fc1 = nn.Linear(784, 512)

        # Hidden layer: 512 input channels, 256 output channels
        self.fc2 = nn.Linear(512, 256)

        # Output layer: 256 input channels, 10 output channels (for 10 classes)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        # Flatten the input from a 28x28 image to a 784-dim vector
        x = x.view(-1, 784)

        # Apply ReLU activation function after first and second fully connected layers
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # Apply Softmax activation function after third fully connected layer
        x = F.softmax(self.fc3(x), dim=1)

        return x

**Hybrid model with both fully layers and convulutional layers**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        # Convolutional layer: 1 input channel (grayscale image), 6 output channels, 3x3 kernel
        # Bias is included by default
        self.conv1 = nn.Conv2d(1, 6, 3)

        # Convolutional layer: 6 input channels, 16 output channels, 3x3 kernel
        # Bias is included by default
        self.conv2 = nn.Conv2d(6, 16, 3)

        # Fully connected layer: 16*6*6 input channels (from the conv2 output), 120 output channels
        # Bias is included by default
        self.fc1 = nn.Linear(16 * 6 * 6, 120)

        # Fully connected layer: 120 input channels, 84 output channels
        # Bias is included by default
        self.fc2 = nn.Linear(120, 84)

        # Output layer: 84 input channels, 10 output channels (for 10 classes)
        # Bias is included by default
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Apply ReLU activation function after each convolutional layer
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        # Flatten the tensor output from the convolutional layers
        x = x.view(-1, 16 * 6 * 6)

        # Apply ReLU activation function after each fully connected layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        # Apply Softmax activation function after output layer
        x = F.softmax(self.fc3(x), dim=1)

        return x

**Model made only from convulutional layers**

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        # Convolutional layer: 1 input channel (grayscale image), 6 output channels, 3x3 kernel
        self.conv1 = nn.Conv2d(1, 6, 3)

        # Convolutional layer: 6 input channels, 16 output channels, 3x3 kernel
        self.conv2 = nn.Conv2d(6, 16, 3)

        # Convolutional layer: 16 input channels, 32 output channels, 3x3 kernel
        self.conv3 = nn.Conv2d(16, 32, 3)

        # Convolutional layer: 32 input channels, 64 output channels, 3x3 kernel
        self.conv4 = nn.Conv2d(32, 64, 3)

        # Output layer: 64 input channels, 10 output channels (for 10 classes), 1x1 kernel
        self.conv5 = nn.Conv2d(64, 10, 1)

    def forward(self, x):
        # Apply ReLU activation function after each convolutional layer
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))

        # Apply Softmax activation function after output layer
        x = F.softmax(self.conv5(x), dim=1)

        # Flatten the tensor output from the convolutional layers
        x = x.view(x.size(0), -1)

        return x

# Model Training

Testing for CUDA

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)



cuda


Model made with only Conv

In [ ]:
# Create an instance of the ConvNet model (defined elsewhere)
model = ConvNet().to(device)

# Define the optimizer (Adam optimizer with learning rate 0.001)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function (CrossEntropyLoss for multi-class classification)
loss_fn = nn.CrossEntropyLoss()

def train(epoch):
  model.train()
  for batch_idx, (data, target) in enumerate(loaders['train']):
    # Move data and target tensors to the chosen device (CPU or GPU)
    data, target = data.to(device), target.to(device)

    # Zero the gradients
    optimizer.zero_grad()

    #Perform a forward pass through the model to get predictions
    output = model(data)

    #Calculate the loss based on prediction and target labels
    loss = loss_fn(output, target)

    #Backpropagate the loss to calculate gradients for each parameter
    loss.backward()

    # Update model parameters based on the calculated gradients using the optimizer
    optimizer.step()
    if batch_idx % 100 == 0:

      # Print training progress information every 100 batches
      print (f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx / len(loaders["train"]):.0f}%)]\tLoss: {loss.item():.6f}')

def test():
  model.eval() # Set model to evaluation mode

  test_loss = 0
  correct = 0

   # Disable gradient calculation for efficiency during testing
  with torch.no_grad():
    for data, target in loaders['test']:
      """
      Iterates over batches of data from the 'test' dataloader.

      Args:
          data: A batch of test images.
          target: A batch of corresponding labels for the test images.
      """

      # Move data and target tensors to the chosen device (CPU or GPU)
      data, target = data.to(device), target.to(device)

      # Get predictions from the model
      output = model(data)

      # Accumulate the loss for the entire test set
      test_loss += loss_fn(output, target)

      # Get the predicted class with the highest probability for each image
      pred = output.max(1, keepdim=True)[1]

      # Calculate the number of correct predictions
      correct += pred.eq(target.view_as(pred)).sum().item()

  # Calculate average test loss
  test_loss /= len(loaders['test'].dataset)

  # Print accuracy and loss for the entire test set
  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(loaders['test'].dataset),
    100. * correct / len(loaders['test'].dataset)))




In [ ]:
for epoch in range(1, 5):
  train(epoch)
  test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 8.304434


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

model.eval()

data, target = test_data[3]

data = data.unsqueeze(0)

output = model(data)

pred = output.max(1, keepdim=True)[1]

print(pred)

image = data.squeeze().numpy()

plt.imshow(image, cmap='gray')
plt.show

NameError: name 'model' is not defined

##KNN##

In [ ]:
from sklearn.model_selection import KFold
import numpy as np # Import numpy

# Define KFold object
num_folds = min(5, len(data))  # Adjust the number of folds to be at most the number of samples
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Loop over folds
for train_index, test_index in kfold.split(data):
  # Create training and validation sets from indices
  X_train_fold, X_val_fold = np.array(data)[train_index], np.array(target)[test_index] # Convert data to a NumPy array for proper indexing
  y_train_fold, y_val_fold = np.array(target)[train_index], np.array(target)[test_index] # Convert target to a NumPy array for proper indexing


  # Train the model on the training fold data
  model = ConvNet().to(device)  # Re-initialize model for each fold
  optimizer = optim.Adam(model.parameters(), lr=0.001)
  loss_fn = nn.CrossEntropyLoss()

  def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train_fold']):
      data, target = data.to(device), target.to(device)
      optimizer.zero_grad()
      output = model(data)
      loss = loss_fn(output, target)
      loss.backward()
      optimizer.step()
      if batch_idx % 100 == 0:
        print (f'Train Fold {train_index+1} Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train_fold"].dataset)} ({100. * batch_idx / len(loaders["train_fold"]):.0f}%)]\tLoss: {loss.item():.6f}')

  def test(model, data_loader, name):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
      for data, target in data_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += loss_fn(output, target)
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(data_loader.dataset)
    print('\n{} set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      name, test_loss, correct, len(data_loader.dataset),
      100. * correct / len(data_loader.dataset)))

  # Train the model
  for epoch in range(1, 5):
    train(epoch)

  # Evaluate the model on the validation fold data
  test(model, loaders['val_fold'], 'Val')

# After iterating through all folds, you can calculate average metrics across folds


ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.

In [ ]:
from sklearn.model_selection import KFold
from torch.utils.data import DataLoader, Subset
import torch.optim as optim

# Define the number of splits
n_splits = 10

# Create a KFold object
kfold = KFold(n_splits=n_splits, shuffle=True)

# Convert the train_data to a tensor
train_data_tensor = torch.tensor(train_data.data.numpy())

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Loop through the indices returned by kfold.split()
for fold, (train_ids, test_ids) in enumerate(kfold.split(train_data_tensor)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    # Create Subset of the data for this fold
    train_subs = Subset(train_data, train_ids)
    test_subs = Subset(train_data, test_ids)

    # Define data loaders for training and testing data in this fold
    trainloader = DataLoader(train_subs, batch_size=100, shuffle=True)
    testloader = DataLoader(test_subs, batch_size=100, shuffle=True)

    # Init the neural network
    network = FullyConnectedNet()

    # Initialize optimizer
    optimizer = optim.Adam(network.parameters(), lr=0.001)

    # Run the training loop for defined number of epochs
    for epoch in range(0, 10):

        # Print epoch
        print(f'Starting epoch {epoch+1}')

        # Set current loss value
        current_loss = 0.0

        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):

            # Get inputs
            inputs, targets = data

            # Zero the gradients
            optimizer.zero_grad()

            # Perform forward pass
            outputs = network(inputs)

            # Compute loss
            loss = criterion(outputs, targets)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Print statistics
            current_loss += loss.item()
            if i % 500 == 499:
                print('Loss after mini-batch %5d: %.3f' %
                      (i + 1, current_loss / 500))
                current_loss = 0.0

    # Process is complete.
    print('Training process has finished.')